This notebook was used to produce Table 1 of
>Anton Osokin, Anatole Chessel, Rafael E. Carazo Salas, Federico Vaggi<br>
GANs for Biological Image Synthesis<br>
In ICCV, 2017

In [1]:
import os
import re
import numpy as np

In [2]:
# parameters of the experiment
num_diter = 5000

log_path = 'logs_eval_8class_real_vs_real'
eval_methods = ['WGAN-GP']
classes_of_interest = ['Alp14', 'Arp3', 'Cki2', 'Mkh1', 'Sid2', 'Tea1', 'Fim1', 'Tea4']

file_template = 'seed{0}_split{1}_eval{2}_iter{3}_classes{4}{5}.txt'

num_splits = 10
rand_seed = 0

In [3]:
def read_last_line(file_name):
    with open(file_name) as f:
        return(list(f)[-1])
    
def get_score(file_name):
    score = float('nan')
    try:
        last_line = read_last_line(file_name)
        numeric_const_pattern = "[-+]?(?:(?:\d*\.\d+)|(?:\d+\.?))(?:[Ee][+-]?\d+)?"
        score_pattern = 'equals\s*(%s)' % numeric_const_pattern
        match = re.search(score_pattern, last_line)
        if match:
            score = float(match.group(1))
    except:
        print('Could not extract value from {0}'.format(file_name))
    return score

In [4]:
mean_all = np.zeros((len(eval_methods), len(classes_of_interest), len(classes_of_interest)))
std_all = np.zeros((len(eval_methods), len(classes_of_interest), len(classes_of_interest)))
for i_eval, eval_method in enumerate(eval_methods):
    for i_cl_a, class_a in enumerate(classes_of_interest):
        for i_cl_b, class_b in enumerate(classes_of_interest):
            split_data = np.zeros(num_splits)
            for i_split in range(num_splits):
                file_name = os.path.join(log_path, file_template)
                file_name = file_name.format(rand_seed, i_split, eval_method, num_diter, class_a, class_b)
                score = get_score(file_name)
                split_data[i_split] = score
        
            mean_all[i_eval, i_cl_a, i_cl_b] = np.nanmean(split_data, axis=0)
            std_all[i_eval, i_cl_a, i_cl_b] = np.nanstd(split_data, axis=0)


In [5]:
def do_table_latex(mean, std, classes_of_interest):
    n_cl = len(classes_of_interest)
    for cl in classes_of_interest:
        print(' & \\protein{{{0}}}'.format(cl), end=' ')
    print('\\\\')
    for i in range(n_cl):
        print('\\protein{{{0}}}'.format(classes_of_interest[i]), end=' ')
        gray_col = np.argmin(mean[i])
        for j in range(n_cl):
            if j != gray_col:
                gray_cell = ''
            else:
                gray_cell = '\\cellcolor{gray!25}'
            print('& {2}{0:0.1f} $\pm$ {1:0.1f}'.format(mean[j, i], std[j, i], gray_cell),end=' ')
                    
        print('\\\\')
                
def do_table(mean, std, classes_of_interest):
    n_cl = len(classes_of_interest)
    for cl in classes_of_interest:
        print('\t{0}'.format(cl), end=' ')
    print('')
    for i in range(n_cl):
        print('{0}'.format(classes_of_interest[i]), end=' ')
        for j in range(n_cl):
            print('\t {0:0.1f}\u00B1{1:0.1f}'.format(mean[i, j], std[i, j]),end=' ')
                    
        print('')

In [6]:
do_latex = False
for i_eval in range(len(eval_methods)):
    print('Evaluation with', eval_methods[i_eval])
    if do_latex:
        do_table_latex(mean_all[i_eval], std_all[i_eval], classes_of_interest)
    else:
        do_table(mean_all[i_eval], std_all[i_eval], classes_of_interest)
    print('')

Evaluation with WGAN-GP
	Alp14 	Arp3 	Cki2 	Mkh1 	Sid2 	Tea1 	Fim1 	Tea4 
Alp14 	 0.1±0.2 	 14.4±0.2 	 8.6±0.2 	 12.3±0.4 	 9.0±0.3 	 11.3±0.3 	 16.3±0.2 	 9.7±0.6 
Arp3 	 12.5±0.3 	 0.8±0.4 	 15.9±0.3 	 12.2±0.6 	 19.5±0.4 	 11.5±0.5 	 2.8±0.3 	 15.8±0.7 
Cki2 	 8.1±0.3 	 16.2±0.2 	 -0.2±0.3 	 13.6±0.3 	 11.8±0.5 	 15.9±0.3 	 18.4±0.2 	 14.0±0.9 
Mkh1 	 12.5±0.5 	 11.5±0.4 	 13.7±0.4 	 -0.2±0.4 	 13.4±0.9 	 14.4±0.6 	 14.5±0.3 	 13.9±0.9 
Sid2 	 9.5±0.2 	 20.5±0.3 	 12.0±0.3 	 12.4±0.6 	 -0.6±0.3 	 13.1±0.1 	 23.4±0.3 	 6.2±0.4 
Tea1 	 10.9±0.3 	 13.2±0.2 	 15.8±0.3 	 13.3±0.6 	 12.6±0.3 	 -0.1±0.4 	 15.1±0.2 	 5.9±0.3 
Fim1 	 15.6±0.3 	 3.7±0.2 	 18.5±0.4 	 15.1±0.5 	 23.9±0.4 	 14.5±0.5 	 -0.2±0.3 	 19.5±0.7 
Tea4 	 11.4±0.3 	 18.3±0.3 	 16.0±0.5 	 14.9±0.8 	 7.7±0.6 	 6.9±0.5 	 20.8±0.5 	 -0.5±0.7 

